In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import urllib.request
from fake_useragent import UserAgent

In [2]:
def get_url(category, page):
       
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.maximize_window()
        driver.get(f"https://www.ozon.ru/category/{category}/?page={page}")
        unique_items = set()
        while True:
            item = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CLASS_NAME, 'tile-hover-target.ip.i0p')))
            H = driver.execute_script('return document.body.scrollHeight;')
            h = 0
            while True :
                h += 300
                if h >= H :
                    break

                for element in WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'tile-hover-target.ip.i0p'))):
                    if element.get_attribute('href') not in unique_items:
                        yield element.get_attribute('href')
                    unique_items.add(element.get_attribute('href'))
                driver.execute_script("window.scrollTo(0, {});".format(h))
                time.sleep(0.1)
            try:
                button = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH,"//button[contains(@class,'Buttonreact__StyledButton')][./i[contains(.,'arrow_forward')]]")))
                driver.execute_script('arguments[0].click();',button)
                WebDriverWait(driver,3).until(EC.staleness_of(item))
            except Exception as e:
                return
        driver.quit()


In [15]:
def get_item(category, page):
    for url_item in get_url(category, page):

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.maximize_window()
        driver.get(url_item)
      

        driver.implicitly_wait(3)

        try:
            title = driver.find_element(By.TAG_NAME, 'h1').text
        except:
            title = ''

        try:
            article = driver.find_element(By.CLASS_NAME, 'k3p.p3k').text
            article = article.replace('Код:', '')
            article = article.replace(' ', '')
        except:
            article = ''

        try:
            price = driver.find_element(By.CLASS_NAME, 'ol0').text
            price = price.replace(' ', '')
            price = price.replace('₽', '')
            price = price.replace('\u2009', '')
        except:
            price = ''


        try:
            cat = driver.find_element(By.CLASS_NAME, 'e4k').text.split('\n')
            try:
                cat_1 = cat[-3]
            except:
                cat_1 = ''
            try:
                cat_2 = cat[-2]
            except:
                cat_2 = ''
            try:
                cat_3 = cat[-1]
            except:
                cat_3 = ''
        except:
            cat_1 = ''
            cat_2 = ''
            cat_3 = ''
        
        try:
            desc_short = driver.find_element(By.CLASS_NAME,'vj8').text
            desc_short = desc_short.replace('\n', ' ')
        except:
            desc_short = ''
        
        try:
            myLink = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.LINK_TEXT, "Перейти к описанию")))
            driver.execute_script('arguments[0].click();',myLink)
        except:
            pass
        

        try:
            desc = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.ID, 'section-description')))
            desc = driver.find_element(By.ID, 'section-description').text
            desc = desc.replace('О товаре', '')
            desc = desc.replace('\n', ' ')
        except:
            desc = ''

        try:
            char = driver.find_element(By.ID, 'section-characteristics').text
            char = char.replace('\n', ' ')
            char = char.replace('Характеристики', '')
            #char = char.replace('Все характеристики', '')
        except:
            char = ''

        driver.quit()

        yield title, article,price, cat_1, cat_2, cat_3, desc, desc_short, char

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3', 
              'description', 'desc_short','caracteristics']

df = pd.DataFrame(columns = df_columns)

In [5]:
def to_csv(category, page):

    for item in get_item(category, page):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
           
    df.to_csv('ozon_new2.csv')

In [6]:
for i in range(6, 7):
    to_csv('begovye-dorozhki-11691', i)
    print(f'Log: page {i} saved...')

Log: page 6 saved...


In [9]:
for i in range(7, 15):
    to_csv('begovye-dorozhki-11691', i)
    print(f'Log: page {i} saved...')

Log: page 7 saved...
Log: page 8 saved...
Log: page 9 saved...
Log: page 10 saved...
Log: page 11 saved...
Log: page 12 saved...
Log: page 13 saved...
Log: page 14 saved...


In [11]:
for i in range(15, 30):
    to_csv('begovye-dorozhki-11691', i)
    print(f'Log: page {i} saved...')

Log: page 15 saved...
Log: page 16 saved...
Log: page 17 saved...
Log: page 18 saved...
Log: page 19 saved...
Log: page 20 saved...
Log: page 21 saved...
Log: page 22 saved...
Log: page 23 saved...
Log: page 24 saved...
Log: page 25 saved...
Log: page 26 saved...
Log: page 27 saved...
Log: page 28 saved...
Log: page 29 saved...


In [14]:
df

,title,article,price,cat_1,cat_2,cat_3,description,desc_short,caracteristics
0,Беговая дорожка Orlauf Hawk A,,60402,Кардиотренажеры,Беговые дорожки,Orlauf,-45% 550 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,"Габариты и вес Ширина бегового полотна, см 45..."
1,"Беговая дорожка для дома Koenigsmann T1.0, скл...",,28286,Кардиотренажеры,Беговые дорожки,Koenigsmann,-45% 1697 бонусов Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,
2,Беговая дорожка электрическая Orlauf Stellar /...,,42673,Кардиотренажеры,Беговые дорожки,Orlauf,-46% 800 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,
3,Беговая дорожка Domsen Fitness F-130,,22278,Кардиотренажеры,Беговые дорожки,Domsen Fitness,-33% 500 баллов за отзыв 1113 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Кол-во программ 15 Страна-изготовитель ...
4,Беговая дорожка Domsen Fitness F-125,,21604,Кардиотренажеры,Беговые дорожки,Domsen Fitness,−22% 500 баллов за отзыв 1080 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,
...,...,...,...,...,...,...,...,...,...
1074,Беговая дорожка электрическая ES-00003 (139 х ...,,19835,Кардиотренажеры,Беговые дорожки,SportElite,,Тип Беговая дорожка Класс тренажера Домашний М...,
1075,"Беговая дорожка VictoryFit VF-730, электрическ...",,55834,Кардиотренажеры,Беговые дорожки,VictoryFit,,Тип Беговая дорожка Класс тренажера Домашний М...,
1076,Беговая дорожка Body Sculpture BT-2760B-H,,18933,Кардиотренажеры,Беговые дорожки,Body Sculpture,,Тип Беговая дорожка Класс тренажера Домашний М...,
1077,Беговая дорожка Orlauf Falcon A,,51762,Кардиотренажеры,Беговые дорожки,Orlauf,,Тип Беговая дорожка Класс тренажера Домашний М...,


In [13]:
for i in range(30, 36):
    to_csv('begovye-dorozhki-11691', i)
    print(f'Log: page {i} saved...')

Log: page 30 saved...
Log: page 31 saved...
Log: page 32 saved...
Log: page 33 saved...
Log: page 34 saved...
Log: page 35 saved...


In [16]:
for i in range(36, 41):
    to_csv('begovye-dorozhki-11691', i)
    print(f'Log: page {i} saved...')

Log: page 36 saved...
Log: page 37 saved...
Log: page 38 saved...
Log: page 39 saved...
Log: page 40 saved...


In [17]:
df

,title,article,price,cat_1,cat_2,cat_3,description,desc_short,caracteristics
0,Беговая дорожка Orlauf Hawk A,,60402,Кардиотренажеры,Беговые дорожки,Orlauf,-45% 550 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,"Габариты и вес Ширина бегового полотна, см 45..."
1,"Беговая дорожка для дома Koenigsmann T1.0, скл...",,28286,Кардиотренажеры,Беговые дорожки,Koenigsmann,-45% 1697 бонусов Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,
2,Беговая дорожка электрическая Orlauf Stellar /...,,42673,Кардиотренажеры,Беговые дорожки,Orlauf,-46% 800 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,
3,Беговая дорожка Domsen Fitness F-130,,22278,Кардиотренажеры,Беговые дорожки,Domsen Fitness,-33% 500 баллов за отзыв 1113 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Кол-во программ 15 Страна-изготовитель ...
4,Беговая дорожка Domsen Fitness F-125,,21604,Кардиотренажеры,Беговые дорожки,Domsen Fitness,−22% 500 баллов за отзыв 1080 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,
...,...,...,...,...,...,...,...,...,...
1253,Беговая дорожка электрическая ES-00003 (139 х ...,Кодтовара:494439015,19835,Кардиотренажеры,Беговые дорожки,SportElite,Описание Домашняя беговая дорожка ТМ SportElit...,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Страна-изготовитель Китай Основные Клас...
1254,Беговая дорожка Everyfit 74209,Кодтовара:993936238,33199,Кардиотренажеры,Беговые дорожки,Everyfit,,Тип Беговая дорожка Класс тренажера Домашний М...,
1255,Беговая дорожка Body Sculpture BT-2760B-H,Кодтовара:321483489,18933,Кардиотренажеры,Беговые дорожки,Body Sculpture,,Тип Беговая дорожка Класс тренажера Домашний М...,
1256,Беговая дорожка LAUFSTEIN EXPLORER AUTOMATIK /...,Кодтовара:356651344,64935,Кардиотренажеры,Беговые дорожки,Laufstein,,Тип Беговая дорожка Класс тренажера Домашний М...,


In [26]:
df = df.drop_duplicates()
df

,title,article,price,cat_1,cat_2,cat_3,description,desc_short,caracteristics
0,Беговая дорожка Orlauf Hawk A,,60402,Кардиотренажеры,Беговые дорожки,Orlauf,-45% 550 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,"Габариты и вес Ширина бегового полотна, см 45..."
1,"Беговая дорожка для дома Koenigsmann T1.0, скл...",,28286,Кардиотренажеры,Беговые дорожки,Koenigsmann,-45% 1697 бонусов Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,
2,Беговая дорожка электрическая Orlauf Stellar /...,,42673,Кардиотренажеры,Беговые дорожки,Orlauf,-46% 800 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,
3,Беговая дорожка Domsen Fitness F-130,,22278,Кардиотренажеры,Беговые дорожки,Domsen Fitness,-33% 500 баллов за отзыв 1113 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Кол-во программ 15 Страна-изготовитель ...
4,Беговая дорожка Domsen Fitness F-125,,21604,Кардиотренажеры,Беговые дорожки,Domsen Fitness,−22% 500 баллов за отзыв 1080 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,
...,...,...,...,...,...,...,...,...,...
1253,Беговая дорожка электрическая ES-00003 (139 х ...,Кодтовара:494439015,19835,Кардиотренажеры,Беговые дорожки,SportElite,Описание Домашняя беговая дорожка ТМ SportElit...,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Страна-изготовитель Китай Основные Клас...
1254,Беговая дорожка Everyfit 74209,Кодтовара:993936238,33199,Кардиотренажеры,Беговые дорожки,Everyfit,,Тип Беговая дорожка Класс тренажера Домашний М...,
1255,Беговая дорожка Body Sculpture BT-2760B-H,Кодтовара:321483489,18933,Кардиотренажеры,Беговые дорожки,Body Sculpture,,Тип Беговая дорожка Класс тренажера Домашний М...,
1256,Беговая дорожка LAUFSTEIN EXPLORER AUTOMATIK /...,Кодтовара:356651344,64935,Кардиотренажеры,Беговые дорожки,Laufstein,,Тип Беговая дорожка Класс тренажера Домашний М...,


In [35]:
df['description_new'] = df['description'].apply(lambda x: '' if str(x).find('%') != -1 else x)
df

C:\Users\Kate\AppData\Local\Temp\ipykernel_14772\2608417884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description_new'] = df['description'].apply(lambda x: '' if str(x).find('%') != -1 else x)


,title,article,price,cat_1,cat_2,cat_3,description,desc_short,caracteristics,description_new
0,Беговая дорожка Orlauf Hawk A,,60402,Кардиотренажеры,Беговые дорожки,Orlauf,-45% 550 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,"Габариты и вес Ширина бегового полотна, см 45...",
1,"Беговая дорожка для дома Koenigsmann T1.0, скл...",,28286,Кардиотренажеры,Беговые дорожки,Koenigsmann,-45% 1697 бонусов Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,,
2,Беговая дорожка электрическая Orlauf Stellar /...,,42673,Кардиотренажеры,Беговые дорожки,Orlauf,-46% 800 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,,
3,Беговая дорожка Domsen Fitness F-130,,22278,Кардиотренажеры,Беговые дорожки,Domsen Fitness,-33% 500 баллов за отзыв 1113 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Кол-во программ 15 Страна-изготовитель ...,
4,Беговая дорожка Domsen Fitness F-125,,21604,Кардиотренажеры,Беговые дорожки,Domsen Fitness,−22% 500 баллов за отзыв 1080 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,,
...,...,...,...,...,...,...,...,...,...,...
1253,Беговая дорожка электрическая ES-00003 (139 х ...,Кодтовара:494439015,19835,Кардиотренажеры,Беговые дорожки,SportElite,Описание Домашняя беговая дорожка ТМ SportElit...,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Страна-изготовитель Китай Основные Клас...,Описание Домашняя беговая дорожка ТМ SportElit...
1254,Беговая дорожка Everyfit 74209,Кодтовара:993936238,33199,Кардиотренажеры,Беговые дорожки,Everyfit,,Тип Беговая дорожка Класс тренажера Домашний М...,,
1255,Беговая дорожка Body Sculpture BT-2760B-H,Кодтовара:321483489,18933,Кардиотренажеры,Беговые дорожки,Body Sculpture,,Тип Беговая дорожка Класс тренажера Домашний М...,,
1256,Беговая дорожка LAUFSTEIN EXPLORER AUTOMATIK /...,Кодтовара:356651344,64935,Кардиотренажеры,Беговые дорожки,Laufstein,,Тип Беговая дорожка Класс тренажера Домашний М...,,


In [37]:
df[(df['description'] != '') & (df['description_new'] == '')]

,title,article,price,cat_1,cat_2,cat_3,description,desc_short,caracteristics,description_new
0,Беговая дорожка Orlauf Hawk A,,60402,Кардиотренажеры,Беговые дорожки,Orlauf,-45% 550 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,"Габариты и вес Ширина бегового полотна, см 45...",
1,"Беговая дорожка для дома Koenigsmann T1.0, скл...",,28286,Кардиотренажеры,Беговые дорожки,Koenigsmann,-45% 1697 бонусов Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,,
2,Беговая дорожка электрическая Orlauf Stellar /...,,42673,Кардиотренажеры,Беговые дорожки,Orlauf,-46% 800 баллов за отзыв Цена что надо,Тип Беговая дорожка Класс тренажера Домашний М...,,
3,Беговая дорожка Domsen Fitness F-130,,22278,Кардиотренажеры,Беговые дорожки,Domsen Fitness,-33% 500 баллов за отзыв 1113 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Кол-во программ 15 Страна-изготовитель ...,
4,Беговая дорожка Domsen Fitness F-125,,21604,Кардиотренажеры,Беговые дорожки,Domsen Fitness,−22% 500 баллов за отзыв 1080 бонусов,Тип Беговая дорожка Класс тренажера Домашний М...,,
...,...,...,...,...,...,...,...,...,...,...
1148,"Беговая дорожка Proxima Felicia II, Арт. PROT-...",Кодтовара:942469155,54990,Кардиотренажеры,Беговые дорожки,ProXima,Описание Электрическая беговая дорожка Proxima...,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Кол-во программ 12 Страна-изготовитель ...,
1151,Беговая дорожка UNIXFIT Беговая дорожка для до...,Кодтовара:478379469,55890,Кардиотренажеры,Беговые дорожки,UNIXFIT,Описание В комплекте защитный коврик UNIX под ...,Тип Беговая дорожка Класс тренажера Домашний М...,Общие Кол-во программ 15 Макс. угол наклона п...,
1156,Беговая дорожка UNIXFIT Беговая дорожка UNIXFI...,Кодтовара:660500615,93890,Кардиотренажеры,Беговые дорожки,UNIXFIT,Описание Беговые дорожки серии MX разработаны ...,Тип Беговая дорожка Класс тренажера Домашний М...,"Габариты и вес Ширина бегового полотна, см 48...",
1170,Беговая дорожка GENAU Walker Pro-X,Кодтовара:563583172,23890,Кардиотренажеры,Беговые дорожки,GENAU,Описание Самый компактная беговая дорожка в ми...,Тип Беговая дорожка Класс тренажера Домашний М...,"Габариты и вес Ширина бегового полотна, см 42...",
